In [1]:
### aqui é para os testes 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups

### Isso é para a classe do professor Thiago
from sklearn.base import BaseEstimator,TransformerMixin
from gensim.matutils import Sparse2Corpus, corpus2dense
from gensim.models.wrappers import LdaMallet
from gensim.corpora.dictionary import Dictionary
import numpy as np
from ldamallet import LdaMalletHandler
import pandas as pd

In [2]:
path = '/home/anasofia/Documents/Projetos/AILab/ALEI-1a/dados/4a_secao/ds.parquet.gzip'

docs = pd.read_parquet(path)

In [3]:
vect = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.7, 
                                min_df = 0.1)

In [4]:
X = vect.fit_transform(docs['text'])

In [5]:
path_to_mallet_binary = "/home/anasofia/Mallet/bin/mallet"

In [6]:
ldamodel = LdaMalletHandler(n_components=20,
                            mallet_path=path_to_mallet_binary,
                            iterations=100,
                            vectorizer=vect)

In [7]:
a, b = ldamodel.vect2gensim(vect, X)

In [8]:
model = LdaMallet(path_to_mallet_binary,
          iterations=100,
          corpus=a,
          num_topics=20,
          id2word=b)

Mallet LDA: 20 topics, 5 topic bits, 11111 topic mask
Data loaded.
max tokens: 27036
total tokens: 11230298
<10> LL/token: -9.46636
<20> LL/token: -8.1142
<30> LL/token: -7.88855
<40> LL/token: -7.83948

0	2.5	imposto renda bahia fonte salvador incidencia brasil pagina juros fone fax andar comercio rendimentos indenizatoria estadual franca retido receita retencao 
1	2.5	manaus zona franca pis cofins zfm exportacao mercadorias decreto fiscais vendas receitas produtos operacoes empresas comercio area dentro livre incentivos 
2	2.5	taxa cento retencao poder servicos exercicio reais cada administracao obrigacoes servico publica limites serao utilizacao provisoria minimo acordo atividades controle 
3	2.5	rua www tel adv fax fone sala paulo bairro brasil mail avenida centro advogados andar boa liminar telefone rodrigues pena 
4	2.5	agravante tutela agravada urgencia antecipacao recursal brasilia dano suspensivo advogados requisitos risco doc suspensao perigo probabilidade bloco interposicao 

In [9]:
ldamodel.model = model

In [10]:
from dfrbrowser import dfrBrowserConverter

In [11]:
data = docs['text']
data.to_csv('meta.csv', sep='\t', index=False, header=True)

In [12]:
conv = dfrBrowserConverter(ldamodel, path='dfr-browser/data')
conv.generate_files('aaaaaa', 'BBBBBBBBJDSAFKLDSJAFKJLS', data=data)